In [ ]:
import numpy as np

print("hello")



In [ ]:
from typing import Callable, List
import math

Coord = List[float]
pi = math.pi

def solve_diffeq(
    x0: Coord, v0: Coord, 
    dvdt: Callable[[float, Coord, Coord], Coord], 
    duration: float, dt: float = 0.01
) -> List[List[float]]:
    """
    Solving the differential equation using the Verlet Integration method
    https://en.wikipedia.org/wiki/Verlet_integration

    """


    # simulate first step
    x = x0
    v = v0
    a = dvdt(0, x, v)

    data = [[x, v]]

    for t in np.arange(dt, duration+dt, dt):
        a_new = dvdt(t, x, v)

        x_new = x + v * dt + 0.5 * a * dt ** 2
        v_new = v + (a + a_new) * dt / 2

        x = x_new
        v = v_new
        a = a_new

        data.append([x, v])


    return data

In [ ]:
%matplotlib qt

import math
import matplotlib.pyplot as plt


pi = math.pi

def dvdt(t, x, v):
    # returns [d long dt, d lat dt]
    # x0 = long, x1 = lat
    x0, x1 = x
    v0, v1 = v
    return np.array([
        2 * v0 * v1 * math.tan(x1),
        - v0 * v0 * math.sin(x1) * math.cos(x1)
    ])

    
def simulate_planet(x0, v0):
    duration = 40
    data = solve_diffeq(
        np.array(x0), np.array(v0),
        dvdt,
        duration
    )

    return data

def modlat(lat):
    while lat > pi/2:
        lat -= pi
    while lat < -pi/2:
        lat += pi
        
    return lat

def modlong(long):
    while long > pi:
        long -= 2*pi
    while long < -pi:
        long += 2*pi
    return long


def graph_data(data, acc = 20):
    plt.style.use('fivethirtyeight')

    plt.figure(figsize=(12, 6))
    plt.xlim(-pi, pi)
    plt.ylim(-pi/2, pi/2)

    # unzip data
    xs = [modlong(d[0][0]) for d in data]
    ys = [modlat(d[0][1]) for d in data]

    plt.suptitle('Simulated Plane Geodesic Path on sphere')
    plt.title('velocity = <1/10, 1/10>', fontsize=16)
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')


    plt.scatter(xs[::acc], ys[::acc], c='r', marker='.', zorder=10)
    #plt.plot(xs, ys, linewidth=4)
    plt.tight_layout()
    plt.show()


v0 = [
    0.5,
    0,
]
x0 = [0, 1]

data = simulate_planet(x0, v0)

graph_data(data, 20)

In [ ]:
import math
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.cm as cm

def latlongToCoords(lat: float, long: float, r: float = 1.0):
    return [
        r * math.cos(lat) * math.cos(long),
        r * math.cos(lat) * math.sin(long),
        r * math.sin(lat)
    ]


def dataToSphericalCoords(data):
    return list(map(lambda d: latlongToCoords(d[1], d[0]), data))


def plotSphericalCoords(coords):
    fig = plt.figure(figsize=(12,12))
    ax = plt.axes(projection='3d')
    
    detail = 50
    xs = [d[0] for d in coords]
    ys = [d[1] for d in coords]
    zs = [d[2] for d in coords]
    
    ax.set_xlim([-1, 1])
    ax.set_ylim([-1, 1])
    ax.set_zlim([-1, 1])
    colors = cm.rainbow(np.linspace(0, 1, len(xs[::detail])))
    ax.scatter3D(xs[::detail], ys[::detail], zs[::detail], label='The path on the 3d sphere', color=colors)
    
    
    # also plot the sphere
    xs, ys, zs = createSphereCoords()
    ax.plot_surface(xs, ys, zs, rstride=1, cstride=1, color='c', alpha=0.04, linewidth=0)
    
    ax.set_box_aspect((1, 1, 1))

    plt.show()


def createSphereCoords():
    r = 1
    pi = np.pi
    cos = np.cos
    sin = np.sin
    lats, longs = np.mgrid[-pi/2:pi/2:20j, -pi:pi:20j]
    x = r*cos(lats)*cos(longs)
    y = r*cos(lats)*sin(longs)
    z = r*sin(lats)
    
    return x, y, z
            
            
def positionData(data):
    return list(map(lambda d: d[0], data))

coords = dataToSphericalCoords(positionData(data))
plotSphericalCoords(coords)